# Homework 09
This homework is based on the classification and regression lectures.

## Question 1
#### In the table below, fill in the definition column with a short (no more than two sentence) definition for each vocab word. If it can be summarized by a formula, give the formula. 

| Vocab Word | Definition |
|:--------|:--------|
| **One-hot coding** | **Encode a categorical variable as binary indicator columns with exactly one “1” per row and the rest “0”.** |
| **Feature selection*** | **Choose a subset of features that best predict the target to reduce variance and improve interpretability. Methods include filter, wrapper, and embedded.** |
| **Classifier** | **A model that maps features to discrete class labels, often also producing class probabilities.** |
| **Precision** | **Fraction of predicted positives that are true positives.** |
| **Recall** | **Fraction of actual positives that are correctly identified.** |
| **F1 Score** | **Harmonic mean of precision and recall.** |
| **Parsimonious model** | **The simplest model that achieves adequate predictive performance, using minimal parameters or features.** |
| **Ridge regression** | **Linear regression with an L2 penalty that shrinks coefficients.** |
| **LASSO regression** | **Linear regression with an L1 penalty that drives some coefficients to zero.** |
| **Cross validation** | **Estimate out-of-sample performance by splitting data into K folds, training on K−1 folds, and validating on the held-out fold, then averaging.** |
| **Tree based methods** | **Models that partition the feature space using hierarchical splits (decision trees) and ensembles like random forests and gradient boosting.** |

*Just give the general idea.

## Question 2 
#### a) What shape does a perfect classifier look like on an ROC curve? What about a bad classifier?
#### b) Think about the formula for an F1 score. What does it mean when the F1 score is close to 1? Close to 0?

In [5]:
## A) A perfect classifier’s ROC curve goes straight up the left axis to (0,1) then across the top—area under the curve (AUC) = 1.
## A bad classifier lies along the diagonal from (0,0) to (1,1) with AUC ≈ 0.5, showing random guessing.
## B) F1 = 1 means both precision and recall are high—predictions are accurate and comprehensive.
## F1 ≈ 0 means poor performance—either many false positives or many missed positives.

## Question 3
#### Compare the following aspects of linear vs. logistic regression.
|  | Linear | Logistic |
|:--------|:--------|:--------|
| **Chart Shape** | **straight line** | **sigmoid** |
| **Dependent Variable Type** | **Continuous.** | **Binary categorical {0,1}** |
| **Purpose** (regression or classification) | **regression** | **classification** |
| **Range of output variable** ($y_i$ or $p_i$) | **(−∞,∞)** | **(0,1) (then class via threshold).** |
| **Method*** | **Ordinary least squares (minimize MSE).** | **Maximum likelihood estimation (Bernoulli/logistic loss).** |
| **Example of use** | **Predict house price.** | **Predict churn yes/no.** |  
  
*Meaning ordinary least squares or maximum likelihood estimation

## Question 4
#### Why is it important to train then test our model? How do we do that? (2-3 sentences. Not looking for code, just general explanation).

In [ ]:
## We train on one subset and test on a separate unseen subset to estimate generalization and detect overfitting.
## Split the data into train and test. Fit and tune on the train only (often with k‑fold cross‑validation), then compute final metrics once on the untouched test set.

## Question 5
This question runs through a linear regression example. We want to predict median house value based on the other variables.
#### a) First, load the `housing.csv` data set. Look at the data in some useful way. Why is linear regression appropriate here?

In [ ]:
## Linear regression is appropriate because the target (median_house_value) is continuous. As a baseline, OLS is reasonable after scaling numeric features and dummy‑coding categoricals.

In [4]:
set.seed(123)
df <- read.csv("/mnt/data/housing.csv", stringsAsFactors = FALSE)
stopifnot("median_house_value" %in% names(df))
str(df)
summary(df$median_house_value)
df <- df[complete.cases(df), ]

Warning message in file(file, "rt"):
“cannot open file '/mnt/data/housing.csv': No such file or directory”


ERROR: Error in file(file, "rt"): cannot open the connection


#### b) Scale data and split it 75/25 training/testing. Set seed = 123.

In [5]:
n <- nrow(df)
idx_tr <- sample.int(n, floor(0.75 * n))
train <- df[idx_tr, ]
test  <- df[-idx_tr, ]

ERROR: Error in sample.int(n, floor(0.75 * n)): length(n) == 1L is not TRUE


#### c) Fit the model.

#### d) Make predictions on test data and show them in an actual vs. predicted plot.

#### e) Make a residuals plot.

## Question 6
This question runs through a logistic regression example. We want to predict diabetes diagnosis based on the other variables. 
#### a) First, load the `diabetes.csv` data set. Look at the data in some useful way. Why is logistic regression appropriate here?

#### b) Scale data and split it 75/25 training/testing. Set seed = 123.

#### c) Fit the model.

#### d) Make predictions on test data. Print a table with the number of true positives, false positives, true negatives, false negatives, and accuracy. 

#### e) Fit a LASSO-regularized logistic regression model. Again, set seed = 123. Which variables are the most important (which ones don't go to zero)? How does the LASSO model affect the accuracy?

#### f) Make a plot of actual vs. predicted values for the LASSO model.